In [1]:
# Importing stock ml libraries
import os
import time
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification


In [2]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# Root label (source = ASRS coding forms) : order = by descending frequency
ANOMALY_LABELS = ['Deviation / Discrepancy - Procedural',
                    'Aircraft Equipment',
                    'Conflict',
                    'Inflight Event / Encounter',
                    'ATC Issue',
                    'Deviation - Altitude',
                    'Deviation - Track / Heading',
                    'Ground Event / Encounter',
                    'Flight Deck / Cabin / Aircraft Event',
                    'Ground Incursion',
                    'Airspace Violation',
                    'Deviation - Speed',
                    'Ground Excursion',
                    'No Specific Anomaly Occurred']

In [4]:
def load_data(path, labels, add_other=False):
    loaded_data = pd.read_pickle(path)[0]

    # Drop Anomaly NaN's
    loaded_data = loaded_data.dropna(subset=['Anomaly']).reset_index(drop=True)

    # Convert the 'Anomaly' column to a list of lists
    anomaly_series = loaded_data['Anomaly']
    anomaly_list = anomaly_series.str.split(';').apply(lambda x: [item.strip() for item in x])

    # Initialize a DataFrame to hold the one-hot-encoded anomalies
    anomaly_df = pd.DataFrame(index=loaded_data.index)

    # Populate the DataFrame with one-hot-encoded columns for each prefix
    for prefix in labels:
        anomaly_df[prefix] = anomaly_list.apply(lambda anomalies: any(anomaly.startswith(prefix) for anomaly in anomalies)).astype(int)

    # Add the 'Other' category
    if add_other:
        anomaly_df['Other'] = (anomaly_df.sum(axis=1) == 0).astype(int)

    # Assign the one-hot-encoded anomalies as a new column 'labels' to 'loaded_data'
    loaded_data['labels'] = anomaly_df.apply(lambda row: row.tolist(), axis=1)

    # Now, 'loaded_data' is a DataFrame that includes both the 'text' and 'labels' columns
    loaded_data['text'] = loaded_data["Narrative"]

    # If you want to create a new DataFrame with just 'text' and 'labels':
    final_df = loaded_data[['text', 'labels']]
    return final_df

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')

drop the NaN values in Anomaly?

In [6]:

train_df = load_data("./data/train_data_final.pkl", ANOMALY_LABELS)
train_df

,text,labels
0,I was the pilot flying performing the takeoff....,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,We had 6 shipments of dry ice for the flight; ...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,I have seen a lot of mistakes on every flight ...,"[1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]"
3,It was my first time flying into KEUG and I wa...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,I am writing this report to bring attention to...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
96981,WE WERE ENRTE IN LNAV AT FL310; 30 MI N OF ATL...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
96982,CLRED BY TWR CTL TO CROSS RWY 8R/26L AT TXWY E...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
96983,WHILE WORKING NUMEROUS CVG AND CMH DEPS AT A C...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
96984,ON MIDNIGHT SHIFT; APPROX XA00 LCL TIME; 2 SEC...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [7]:
test_df = load_data("./data/test_data_final.pkl", ANOMALY_LABELS)
test_df

,text,labels
0,Flying into SLC on the DELTA THREE RNAV arriva...,"[1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
1,ORD was on a very busy east flow arrival push....,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,B737-800 was vectored to an ILS Runway 16L app...,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,We were on a 6 mile final when tower cleared a...,"[1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
4,During Climb we Leveled at 17;000 departure sw...,"[1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...
10800,FO was flying a visual approach to runway 26 i...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10801,While assembling a GE C2 transfer gearbox; I n...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10802,Nearing the end of a hot; bumpy four-hour IFR ...,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
10803,On approach gear went down and noticed yellow ...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [8]:
MODEL_NAME = None
MODEL_DIRECTORY = "model_save"


# Sections of configBertTokenizer
# Defining some key variables that will be used later on in the training
BALANCED = False
LAYERS_TO_UNFREEZE = None
LAYERS_TO_UNFREEZE = [8, 9, 10, 11]
# MAX_LEN = 512
MAX_LEN = 4096
TRAIN_EFFECTIVE_BATCH_SIZE = 32
TRAIN_BATCH_SIZE = 16 # 32 Effective size for NASA
ACCUMULATION_STEPS = TRAIN_EFFECTIVE_BATCH_SIZE / TRAIN_BATCH_SIZE
VALID_BATCH_SIZE = 32
EPOCHS = 5 # 5 Epochs for NASA
LEARNING_RATE = 1e-05 * 2 # 0.00002 Rate for NASA

In [9]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text.iloc[index])
        text = " ".join(text.split())

        inputs = self.tokenizer(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets.iloc[index], dtype=torch.float)
        }


In [10]:
class SequenceClassificationModel(torch.nn.Module):
    def __init__(self, model_name='bert-base-uncased', num_labels=15):
        super(SequenceClassificationModel, self).__init__()
        self.original_name = model_name
        self.model_name = model_name.replace("/", "_")
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    def forward(self, ids, mask, token_type_ids):
        output = self.model(ids, attention_mask=mask, token_type_ids=token_type_ids)
        return output.logits
    
    def tokenizer(self):
        return AutoTokenizer.from_pretrained(self.original_name)
    
    def _set_layer_trainable(self, layer, trainable):
        for param in layer.parameters():
            param.requires_grad = trainable

    def _find_and_set_encoder_layers(self, module, layer_nums, trainable):
        if hasattr(module, 'encoder'):
            for layer_num in layer_nums:
                try:
                    self._set_layer_trainable(module.encoder.layer[layer_num], trainable)
                except IndexError:
                    print(f"Layer {layer_num} not found in the encoder.")
            return True
        else:
            for child in module.children():
                if self._find_and_set_encoder_layers(child, layer_nums, trainable):
                    return True
        return False

    def set_trainable_layers(self, layer_nums=None):
        if layer_nums is not None:
            self.model_name += f'{self.model_name}_Unfrozen{layer_nums}'
            
        # Freeze all parameters first
        for param in self.parameters():
            param.requires_grad = False

        # Unfreeze classifier layers
        if hasattr(self.model, 'classifier'):
            self._set_layer_trainable(self.model.classifier, True)

        # Attempt to find and unfreeze encoder layers
        if not self._find_and_set_encoder_layers(self.model, layer_nums or [], True):
            print("Encoder layers not found in the model.")



In [11]:
num_labels = len(test_df.labels[0])

# model = SequenceClassificationModel('bert-base-uncased', num_labels=num_labels)
# model = SequenceClassificationModel('NASA-AIML/MIKA_SafeAeroBERT', num_labels=num_labels)
model = SequenceClassificationModel('allenai/longformer-base-4096', num_labels=num_labels)

model.set_trainable_layers(LAYERS_TO_UNFREEZE)
model.to(device)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassificationModel(
  (model): LongformerForSequenceClassification(
    (longformer): LongformerModel(
      (embeddings): LongformerEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (position_embeddings): Embedding(4098, 768, padding_idx=1)
      )
      (encoder): LongformerEncoder(
        (layer): ModuleList(
          (0-11): 12 x LongformerLayer(
            (attention): LongformerAttention(
              (self): LongformerSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (query_global): Linear(in_features=768, out_features=768, bias=True)
                (

In [12]:
# Creating the dataset and dataloader for the neural network
print("TRAIN Dataset: {}".format(train_df.shape))
print("TEST Dataset: {}".format(test_df.shape))

tokenizer = model.tokenizer()
training_set = CustomDataset(train_df, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_df, tokenizer, MAX_LEN)

TRAIN Dataset: (96986, 2)
TEST Dataset: (10805, 2)


In [13]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 2
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [14]:
torch.cuda.empty_cache()

In [15]:
def binary_accuracy_per_label(y_true, y_pred, threshold=0.5):
    preds = (y_pred >= threshold)
    correct = preds == y_true
    accuracy_per_label = correct.mean(axis=0)
    return accuracy_per_label

def binary_accuracy_averaged(y_true, y_pred, threshold=0.5):
    accuracy_per_label = binary_accuracy_per_label(y_true, y_pred, threshold)
    accuracy_averaged = accuracy_per_label.mean()
    return accuracy_averaged

def custom_classification_report(y_true, y_pred):
    report = metrics.classification_report(y_true, y_pred, output_dict=True, labels=ANOMALY_LABELS, zero_division=0)
    accuracy = binary_accuracy_per_label(y_true, y_pred)
    extended_accuracy_new = np.append(accuracy, [accuracy.mean()] * (len(report) - len(accuracy)))

    updated_report = {}
    for i, class_label in enumerate(report.keys()):
        # Create a new dictionary for the class with binary accuracy
        class_dict = {'binary_accuracy': extended_accuracy_new[i]}
        
        # Merge this dictionary with the existing metrics for the class
        class_dict.update(report[class_label])

        # Update the main report dictionary
        updated_report[class_label] = class_dict

    return updated_report


In [16]:
pos_weight = None
if BALANCED:
    # Compute weights for loss function
    num_labels = len(training_set[0]['targets'])
    pos_num = torch.zeros(num_labels).to(device)
    for _, data in enumerate(training_loader, 0):
        targets = data['targets'].to(device)
        pos_num += torch.sum(targets, axis=0)
    nobs = len(training_loader.dataset)
    pos_weight = (nobs - pos_num) / pos_num

loss_fn = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)  # compute weighted loss for unbalanced dataset
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
metrics_dict = {"Custom Classifcation Report": lambda y_true, y_pred: custom_classification_report(y_true, y_pred)
    # "Binary Accuracy Macro": lambda outputs, targets: binary_accuracy_averaged(targets, outputs, threshold=0.5),
    # "Binary Accuracy per Class": binary_accuracy_per_label,
    # "F1 Score Micro": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='micro', zero_division=1),
    # "F1 Score Macro": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average='macro', zero_division=1),
    # "F1 Scores per Class": lambda y_true, y_pred: metrics.f1_score(y_true, y_pred, average=None, zero_division=1)
}


In [17]:
def save_model(model, epoch, directory='model_save', model_name=None):
    """
    Saves the model state.

    Args:
    model (torch.nn.Module): The model to save.
    epoch (int): The current epoch number.
    file_path (str): Base directory to save the models.
    """
    if model_name is None:
        model_name = model.model_name

    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file_path = os.path.join(directory, f"{model_name}_epoch_{epoch}.pth")

    torch.save(model.state_dict(), file_path)
    print(f'Model saved at {file_path}')


In [18]:
def load_model(model, directory='model_save', model_name=None, epoch=None):
    """
    Loads the model state.

    Args:
    model (torch.nn.Module): The model to load state into.
    file_path (str): Path to the saved model file.
    """
    if model_name is None:
        model_name = model.model_name

    if epoch is None:
        epoch = find_last_saved_epoch(directory, model_name)
        if epoch == -1:
            print("No saved model found.")
            return
    
    file_path = os.path.join(directory, f"{model_name}_epoch_{epoch}.pth")
    if not os.path.exists(file_path):
        print(f"No model file found at {file_path}")
        return

    model.load_state_dict(torch.load(file_path))
    model.to(device)
    print(f'Model loaded from {file_path}')

In [19]:
def find_last_saved_epoch(directory='model_save', model_name=None):
    """
    Finds the last saved epoch number in the specified directory.

    Args:
    file_path (str): The directory where models are saved.

    Returns:
    int: The last saved epoch number. Returns -1 if no saved model is found.
    """
    if model_name is None:
        model_name = model.model_name

    # Check if the directory exists, and create it if it doesn't
    if not os.path.exists(directory):
        return -1

    saved_epochs = []
    for filename in os.listdir(directory):
        if model_name is None or filename.startswith(model_name):
            parts = filename.replace('.pth', '').split('_')
            if parts[-2] == 'epoch':
                try:
                    saved_epochs.append(int(parts[-1]))
                except ValueError:
                    pass
    
    return max(saved_epochs, default=-1)

In [20]:
def process_batch(model, batch_data, device, loss_fn, mode, optimizer=None, accumulate_gradients=False):
    ids = batch_data['ids'].to(device, dtype=torch.long)
    mask = batch_data['mask'].to(device, dtype=torch.long)
    token_type_ids = batch_data['token_type_ids'].to(device, dtype=torch.long)
    targets = batch_data['targets'].to(device, dtype=torch.float)

    if mode == 'train':
        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        loss.backward()
        if not accumulate_gradients:
            optimizer.step()
            optimizer.zero_grad()
    else:
        with torch.no_grad():
            outputs = model(ids, mask, token_type_ids)
            loss = loss_fn(outputs, targets)

    return outputs, targets, loss


In [21]:
def calculate_metrics(metrics_dict, y_true, y_pred):
    results = {}
    labels = anomaly_labels + ["Other"]
    for metric_name, metric_fn in metrics_dict.items():
        if metric_name == "F1 Scores per Class":
            # Calculate F1 score for each class and associate with labels
            f1_scores = metric_fn(y_true, y_pred)
            for i, score in enumerate(f1_scores):
                label = labels[i] if i < len(labels) else f"Class {i}"
                results[f"F1 Score - {label}"] = score
        elif metric_name == "Binary Accuracy per Class":
            bin_acc = metric_fn(y_true, y_pred)
            for i, score in enumerate(bin_acc):
                label = labels[i] if i < len(labels) else f"Class {i}"
                results[f"Binary Accuracy - {label}"] = score
        else:
            results[metric_name] = metric_fn(y_true, y_pred)
    return results

In [22]:
def format_value(val):
    """Helper function to format the value for printing."""
    return f"{val:.4f}" if isinstance(val, (float, np.float64)) else val

def print_metrics_results(metrics_results):
    # First, print scalar values and simple dictionaries
    for metric, value in metrics_results.items():
        if isinstance(value, dict) and not any(isinstance(v, dict) for v in value.values()):
            # Print simple dictionaries on a single line
            dict_values = ", ".join([f"{k}: {format_value(v)}" for k, v in value.items()])
            print(f"{metric}: {dict_values}")
        elif not isinstance(value, dict):
            # Print scalar values
            print(f"{metric}: {format_value(value)}")

    # Then, print nested dictionaries
    for metric, value in metrics_results.items():
        if isinstance(value, dict) and any(isinstance(v, dict) for v in value.values()):
            # Print nested dictionaries
            print(f"\n{metric}:")
            # Find the longest key length for formatting
            max_key_length = max(len(str(k)) for k in value.keys())
            for sub_key, sub_dict in value.items():
                formatted_key = f"{sub_key}:".ljust(max_key_length + 2)
                dict_values = ", ".join([f"{k}: {format_value(v)}" for k, v in sub_dict.items()])
                print(f"  {formatted_key} {dict_values}")


In [23]:
def print_batch_results(mode, epoch, batch, dataset_size, loss, start_time, batch_start_time, batch_size):
    current_time = time.time()
    elapsed_time = current_time - start_time
    batch_time_ms = (current_time - batch_start_time) * 1000

    current = (batch + 1) * batch_size
    epoch_str = f"Epoch: {epoch+1}, " if epoch is not None else ""
    
    print(f"\r{mode.capitalize()} - {epoch_str}Batch: {batch+1} [{current:>5d}/{dataset_size:>5d}], "
          f"Time: {elapsed_time:.0f}s {batch_time_ms:.0f}ms/step, Loss: {loss:>7f}", end="")


In [24]:
def process_batches(mode, model, loader, device, loss_fn, optimizer=None, epoch=None, accumulation_steps=None):
    total_loss = 0.0
    all_targets = []
    all_outputs = []
    start_time = time.time()

    for batch, data in enumerate(loader, 0):
        batch_start_time = time.time()
        
        logits, targets, loss = process_batch(model, data, device, loss_fn, mode, optimizer)
        total_loss += loss.item()

        if mode == 'train':
            if accumulation_steps is not None and (batch + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

        outputs_binary = torch.sigmoid(logits).cpu().detach().numpy() >= 0.5
        targets = targets.cpu().detach().numpy()
        all_outputs.extend(outputs_binary)
        all_targets.extend(targets)

        batch_size = targets.shape[0]
        print_batch_results(mode, epoch, batch, len(loader.dataset), loss.item(), start_time, batch_start_time, batch_size)

    if mode == 'train' and optimizer is not None and accumulation_steps is not None:
        # Ensure any remaining gradients are applied
        optimizer.step()
        optimizer.zero_grad()
    
    all_outputs = np.vstack(all_outputs)
    all_targets = np.vstack(all_targets)

    print()
    avg_loss = total_loss / len(loader)
    return avg_loss, all_outputs, all_targets

In [25]:
def evaluate(model, validation_loader, loss_fn, metrics_dict, device):
    model.eval()
    avg_val_loss, val_outputs, val_targets = process_batches('evaluate', model, validation_loader, device, loss_fn)
    
    metrics_results = calculate_metrics(metrics_dict, val_targets, val_outputs)

    print(f"Evaluation Results:")
    print(f"Average Loss: {avg_val_loss:.4f}")
    print_metrics_results(metrics_results)

    return avg_val_loss, metrics_results


In [26]:
def train(model, epoch, training_loader, validation_loader, optimizer, loss_fn, metrics_dict, device, accumulation_steps=1):
    print(f"Training Epoch {epoch + 1}")

    # Training phase
    model.train()
    if optimizer is not None:
        optimizer.zero_grad()

    avg_train_loss, train_outputs, train_targets = process_batches('train', model, training_loader, device, loss_fn, optimizer, epoch, accumulation_steps)
    print(f"Average Training Loss for Epoch {epoch + 1}: {avg_train_loss:.4f}")

    metrics_results = calculate_metrics(metrics_dict, train_targets, train_outputs)

    print(f"Evaluation Results:")
    print(f"Average Loss: {avg_val_loss:.4f}")
    print_metrics_results(metrics_results)

    # Validation phase
    if validation_loader is not None:
        avg_val_loss, val_metrics_results = evaluate(model, validation_loader, loss_fn, metrics_dict, device)
    else:
        avg_val_loss = None
        val_metrics_results = {}

    return avg_train_loss, avg_val_loss, val_metrics_results


In [27]:
last_saved_epoch = find_last_saved_epoch(directory=MODEL_DIRECTORY, model_name=MODEL_NAME)

start_epoch = last_saved_epoch + 1 if last_saved_epoch != -1 else 0
if last_saved_epoch != -1:
    load_model(model, directory=MODEL_DIRECTORY, model_name=MODEL_NAME, epoch=last_saved_epoch)
    print(f"Loaded model training from epoch {start_epoch}")
else:
    print("No saved model found.")

if start_epoch < EPOCHS:
    print(f"Resuming training from epoch {start_epoch + 1}")

for epoch in range(start_epoch, EPOCHS):
    train_loss, val_loss, val_metrics = train(model, epoch, training_loader, testing_loader, optimizer, loss_fn, metrics_dict, device, accumulation_steps=8)
    save_model(model, epoch, directory=MODEL_DIRECTORY, model_name=MODEL_NAME)
    # Additional epoch-level processing if needed

# Testing phase
avg_test_loss, test_metrics_results = evaluate(model, testing_loader, loss_fn, metrics_dict, device)
print(f"Test Results:")
print(f"Average Loss: {avg_test_loss:.4f}")
print_metrics_results(test_metrics_results)


No saved model found.
Resuming training from epoch 1
Training Epoch 1


OutOfMemoryError: CUDA out of memory. Tried to allocate 290.00 MiB. GPU 0 has a total capacty of 5.80 GiB of which 95.50 MiB is free. Including non-PyTorch memory, this process has 5.69 GiB memory in use. Of the allocated memory 5.19 GiB is allocated by PyTorch, and 377.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF